In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=5
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A2_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A2_018', 'Input_A2_019', 'Input_A1_019', 'Input_A5_016',
       'Input_A6_016'], dtype=object)

## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,5)

train_rmse: 0.008300276103545664
test_rmse: 0.009091755270109353
test_r2: 0.5710755789644553


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,0.0005)

test_rmse_ave: 0.00910124053474811
[0.009709733726552277, 0.00903254648332744, 0.009161053196929878, 0.009695957276419859, 0.008002412294902086, 0.008361853381021645, 0.009260446777294633, 0.009609651649654691, 0.008750687838157823, 0.009428062723220764]


test_r2_ave: 0.5703366009778013
[0.580502821343885, 0.5977265827157261, 0.5915289017398306, 0.5096525327379231, 0.6261797613027048, 0.6306125850546909, 0.5375127791499521, 0.5068904526864819, 0.5995481395999065, 0.5232114534469117]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.04],0.02)

test_rmse_ave: 0.009176759491504463
[0.009890539087653034, 0.009058458204247498, 0.009167258653892296, 0.009821522348946033, 0.008012159458043138, 0.008452595822201415, 0.009326859337512618, 0.00968870684902373, 0.008904617389020276, 0.009444877764504582]


test_r2_ave: 0.5632259881935459
[0.5647344142441564, 0.595415263829341, 0.5909753391193253, 0.49687005230578496, 0.625268559586482, 0.6225519343413466, 0.5308554116567579, 0.4987438050981534, 0.5853358789188333, 0.5215092228352775]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,5,model)

train_rmse: 0.008430247727480519
test_rmse: 0.009470809390685942
test_r2: 0.5339600703664058


## XGB

In [9]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.09,
              booster = 'gbtree', 
              n_estimators = 85, 
              max_depth = 4, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.55,
              reg_alpha =  0,
              reg_lambda = 0.7)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.008837065003264812
[0.009249575666597083, 0.009080898032367389, 0.009076309777001666, 0.009747257208273572, 0.007739830988184788, 0.007863031247109086, 0.00936565923982324, 0.008933484954183593, 0.008048113055456553, 0.009266489863651159]


test_r2_ave: 0.5941379546490498
[0.6193217862545504, 0.5934082871763435, 0.5990509921904466, 0.5044500888430413, 0.65030943168478, 0.6733693001207957, 0.5269439929358045, 0.573842690147869, 0.6612696823484336, 0.5394132947884347]


## RandomForest

In [10]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 90,
            max_depth = 7,
            min_samples_split = 9,
            max_leaf_nodes = 11,
            min_samples_leaf =3,
            random_state = 42) 


In [11]:
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.008925843357390223
[0.009210125256724678, 0.00913248179150185, 0.00920898732491742, 0.009937240180769243, 0.008075117896793299, 0.007625395239402569, 0.009415627147691322, 0.00915181698445304, 0.008390263020983843, 0.009111378730664966]


test_r2_ave: 0.5856896471260756
[0.6225621262697759, 0.5887759042016061, 0.5872431652137791, 0.48494438801992423, 0.6193562459621168, 0.6928137895868074, 0.5218828062346494, 0.5527578001899157, 0.6318565454027938, 0.5547037001793874]


In [12]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=0.0005)
elastic_mod=ElasticNet(alpha=[0.04], l1_ratio=0.02)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.09,
              booster = 'gbtree', 
              n_estimators = 85, 
              max_depth = 4, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.55,
              reg_alpha =  0,
              reg_lambda = 0.7)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 90,
            max_depth = 7,
            min_samples_split = 9,
            max_leaf_nodes = 11,
            min_samples_leaf =3,
            random_state = 42) 
# vote_mod = VotingRegressor([ ('Lasso', lasso_mod),('SVR', svr_mod), ('Elastic', elastic_mod), 
#                             ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
vote_mod = VotingRegressor([ ('Lasso', lasso_mod),('Linear', lr), ('Elastic', elastic_mod), 
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.008894241783856458
[0.009435404851279439, 0.0089522619771371, 0.009025342353005783, 0.009631853714809804, 0.00784518583724515, 0.008007850870916546, 0.009149145570139383, 0.00926077836869627, 0.008409862105422466, 0.009224732189912627]


test_r2_ave: 0.589462322713373
[0.6038720533900617, 0.6048459021859673, 0.6035413548885316, 0.5161148366702217, 0.640724637998576, 0.661226873731953, 0.5485632276128174, 0.5420446857918675, 0.6301346203976239, 0.5435550344661109]


In [23]:

stack_mod = StackingRegressor(regressors=[lasso_mod, elastic_mod,  vote_mod], 
                           meta_regressor=random_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.009011328326770079
[0.009119111577840842, 0.009145963286142717, 0.009133352280681729, 0.009639750149242607, 0.007901533710808052, 0.008140410177431766, 0.009843691824080882, 0.009135625521493766, 0.008193347135432927, 0.00986049760454551]


test_r2_ave: 0.5771490621929096
[0.6299848869075355, 0.5875608987797938, 0.5939954121783628, 0.5153211091405603, 0.6355451290409089, 0.649918165916123, 0.47742109491219176, 0.5543389287595808, 0.6489341032532878, 0.4784708930407523]


In [24]:
ave(skew_data[title],y,0.2,stack_mod,0.5,vote_mod,0.3,xgb_mod)

test_rmse_ave: 0.0088224374905826
[0.009276149690720874, 0.008938974147393254, 0.009002782508739645, 0.00961346977701851, 0.007740310852882915, 0.007912513968530462, 0.009254815980525074, 0.00906335612726718, 0.008168213195189636, 0.009253788657558453]


test_r2_ave: 0.5957338623134948
[0.6171312676298404, 0.6060180849381018, 0.6055208625073383, 0.5179602187855834, 0.6502660691459505, 0.6692453352910285, 0.5380750370800487, 0.5613620412257445, 0.6510846615941832, 0.5406750449371291]
